#**Calculation of VR-Outcome**
Each of VR-outcome is average of outcome from VGG16 and ResNet50. For each of the image in test-set, prediction is performed by averaging outcome of invdividual model. 

In [1]:
#Authentication from GoogleDrive 
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [4]:
#Combining confidence of both VGG and ResNet models into a single file 
import os
import glob
import pandas as pd
vr_no=1                                             #Indicates individual model   

fileList=glob.glob("/content/gdrive/My Drive/MediaEval_2019/predictions/*_pred_"+str(vr_no)+".csv") #Path of prediction by VGG and ResNet
conf=[]                                             #It will store confidence predicted for each image in test-set
imgIds =[]                                          #It will store ids of images available in test-set
for filename in fileList:                           #loop which run for two iterations, one to collect prediction by VGG and another for ResNet
    df=pandas.read_csv(filename,header=None)        #
    imgIds.append(df.iloc[0:,0])                    #read img_id column from predicted file
    conf.append(df.iloc[0:,2])                      #read confidence of prediction for each image from predicted file 
    concatDF=pandas.concat(conf,axis=1)             
    #save prediction by VGG and ResNet in single file
    concatDF.to_csv("/content/gdrive/My Drive/MediaEval_2019/ensemble/V_R_separate_"+str(vr_no)+".csv",index=None,header=None)
df_imgIds=pd.DataFrame(imgIds[0])


In [ ]:
#Calculating average of confidence predicted by VGG and ResNet 
with open("/content/gdrive/My Drive/MediaEval_2019/ensemble/V_R_separate_"+str(vr_no)+".csv", "r") as filename: 
    arr_pred= []                                    #array to store predictions of each VR-outcome 
    for line in filename:
        stripped_line  = line.strip() 
        stripped_line  = stripped_line.split(',')
        line_sum = sum(float(i) for i in stripped_line ) 
        line_average=line_sum/len(stripped_line)   #Store average of prediction from VGG and ResNet   
        single_pred = 0 if line_average<0.5 else 1 #If average of both prediction is less than 0.5 then images relates to 0, otherwise 1                               
        arr_pred.append(single_pred)               #prediction for individual image is appended in an array
#Put prediction in dataframe and save in csv file
df_pred= pd.DataFrame(arr_pred)                    
df_pred.to_csv("/content/gdrive/My Drive/MediaEval_2019/ensemble/VR_outcome_"+str(vr_no)+".csv", mode='a', header=None, index=False)


#**Calculation of Majority Voting**
Outcome of individual VR-outcome is stored in csv file, for each of the image in train-set. Finally, majority voting is applied to produce final prediction of VRBagged-Net. 

In [4]:
#Combining prediction of individual VR-outcome, in one file
import os
import glob
import pandas 
fileList=glob.glob("/content/gdrive/My Drive/MediaEval_2019/ensemble/VR_outcome*.csv")            #List of files, containing prediction of individual VR-outcome
dfList=[]
for filename in fileList: 
    df=pandas.read_csv(filename,header=None)              
    dfList.append(df.iloc[0:,0])                    #read prediction of each VR-outcome 
    concatDF=pandas.concat(dfList,axis=1)           #concate prediction of each vr-outcome, to create ensemble 
    concatDF.to_csv("/content/gdrive/My Drive/MediaEval_2019/ensemble/VR_outcome_ensemble.csv",index=None,header=None)

In [ ]:
#Calculation of majority voting for all VR-outcomes prediction 
import statistics
with open("/content/gdrive/My Drive/MediaEval_2019/ensemble/VR_outcome_ensemble.csv", "r") as filename: 
    majority= [] 
    for line in filename:
        stripped_line  = line.strip() 
        stripped_line  = stripped_line.split(',')
        row_mode = statistics.mode(stripped_line)       #Mode function is used to select majority voting 
        majority.append(row_mode)

In [4]:
#Append majority prediction in the last column of same file 
df_prediction=pd.read_csv('/content/gdrive/My Drive/MediaEval_2019/ensemble/VR_outcome_ensemble.csv', header=None)
df_majority= pd.DataFrame(majority)
print(df_imgIds.size)
result = pd.concat([df_imgIds, df_prediction, df_majority], axis=1) #img_ids, individual predictions and majority voting are combined in result variable
print(result)
result.to_csv('/content/gdrive/My Drive/MediaEval_2019/ensemble/mjority_VR_outcome_ensemble.csv', mode='a', header=None,index=False)